# Assignment 3 - Building a Custom Visualization

---

In this assignment you must choose one of the options presented below and submit a visual as well as your source code for peer grading. The details of how you solve the assignment are up to you, although your assignment must use matplotlib so that your peers can evaluate your work. The options differ in challenge level, but there are no grades associated with the challenge level you chose. However, your peers will be asked to ensure you at least met a minimum quality for a given technique in order to pass. Implement the technique fully (or exceed it!) and you should be able to earn full grades for the assignment.


&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Ferreira, N., Fisher, D., & Konig, A. C. (2014, April). [Sample-oriented task-driven visualizations: allowing users to make better, more confident decisions.](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/Ferreira_Fisher_Sample_Oriented_Tasks.pdf) 
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;In Proceedings of the SIGCHI Conference on Human Factors in Computing Systems (pp. 571-580). ACM. ([video](https://www.youtube.com/watch?v=BI7GAs-va-Q))


In this [paper](https://www.microsoft.com/en-us/research/wp-content/uploads/2016/02/Ferreira_Fisher_Sample_Oriented_Tasks.pdf) the authors describe the challenges users face when trying to make judgements about probabilistic data generated through samples. As an example, they look at a bar chart of four years of data (replicated below in Figure 1). Each year has a y-axis value, which is derived from a sample of a larger dataset. For instance, the first value might be the number votes in a given district or riding for 1992, with the average being around 33,000. On top of this is plotted the 95% confidence interval for the mean (see the boxplot lectures for more information, and the yerr parameter of barcharts).

<br>
<img src="readonly/Assignment3Fig1.png" alt="Figure 1" style="width: 400px;"/>
<h4 style="text-align: center;" markdown="1">  &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Figure 1 from (Ferreira et al, 2014).</h4>

<br>

A challenge that users face is that, for a given y-axis value (e.g. 42,000), it is difficult to know which x-axis values are most likely to be representative, because the confidence levels overlap and their distributions are different (the lengths of the confidence interval bars are unequal). One of the solutions the authors propose for this problem (Figure 2c) is to allow users to indicate the y-axis value of interest (e.g. 42,000) and then draw a horizontal line and color bars based on this value. So bars might be colored red if they are definitely above this value (given the confidence interval), blue if they are definitely below this value, or white if they contain this value.


<br>
<img src="readonly/Assignment3Fig2c.png" alt="Figure 1" style="width: 400px;"/>
<h4 style="text-align: center;" markdown="1">  Figure 2c from (Ferreira et al. 2014). Note that the colorbar legend at the bottom as well as the arrows are not required in the assignment descriptions below.</h4>

<br>
<br>

**Easiest option:** Implement the bar coloring as described above - a color scale with only three colors, (e.g. blue, white, and red). Assume the user provides the y axis value of interest as a parameter or variable.


**Harder option:** Implement the bar coloring as described in the paper, where the color of the bar is actually based on the amount of data covered (e.g. a gradient ranging from dark blue for the distribution being certainly below this y-axis, to white if the value is certainly contained, to dark red if the value is certainly not contained as the distribution is above the axis).

**Even Harder option:** Add interactivity to the above, which allows the user to click on the y axis to set the value of interest. The bar colors should change with respect to what value the user has selected.

**Hardest option:** Allow the user to interactively set a range of y values they are interested in, and recolor based on this (e.g. a y-axis band, see the paper for more details).

---

*Note: The data given for this assignment is not the same as the data used in the article and as a result the visualizations may look a little different.*

In [ ]:
# Use the following data for this assignment:

import pandas as pd
import numpy as np

np.random.seed(12345)

df = pd.DataFrame([np.random.normal(32000,200000,3650), 
                   np.random.normal(43000,100000,3650), 
                   np.random.normal(43500,140000,3650), 
                   np.random.normal(48000,70000,3650)], 
                  index=[1992,1993,1994,1995])

In [ ]:
from matplotlib import pyplot as plt
%matplotlib notebook

In [ ]:
values = df.mean(axis=1)
yerr = 1.96*(df.std(axis=1)/np.sqrt(df.count(axis=1)))

In [ ]:
# Definition of the color used in almost all texts/lines and the x-axis labels
official_color = 'dimgrey'
x_ind = df.index

# Define size and remove chart borders
plt.figure(figsize=(8,4))
plt.axes(frameon=False)

# Plot the bars and customize the xticks labels
barlist = plt.bar(x_ind, values, edgecolor=official_color, linewidth=0.1, color='springgreen',
                  yerr=yerr, error_kw=dict(ecolor=official_color, lw=2, capsize=3, capthick=2, alpha=0.55))
plt.xticks(x_ind, df.index.astype(str))

# Remove ticks from y-axis and change the text colors
plt.tick_params(
                axis='y',
                which='both',
                bottom='off',
                top='off',
                left='off',
                labelbottom='off'
)
plt.tick_params(colors=official_color)
plt.show()

# Function to find the interval of each bar
def which_interval(value, parameter):
    '''value, parameter: value to compare
    Returns: -1, certainly smaller than parameter, 1, certainly bigger than paramenter, otherwise, 0'''
    
    # Safe comparison for float
    if str(value) == str(parameter):
        return 0
    if value > parameter:
        return 1
    return -1
    

# Colors used to bars
colors = ['white', 'tomato', 'cornflowerblue']

In [ ]:
# Function to insert and manage the line inputed by the user
def update_colors(parameter, dic={'line': None, 'annotation': None}):
    # Remove the old line and annotation
    if dic['line']:
        dic['line'].remove()
    if dic['annotation']:
        dic['annotation'].remove()
    
    # Create the new line and annotation with the value inputed
    dic['line'] = plt.axhline(y=parameter, color=official_color, linestyle='-', label=parameter, linewidth=0.8)
    dic['annotation'] = plt.annotate('%0.2f' % parameter, xy=(1, parameter-800), xytext=(8, 0), 
                 xycoords=('axes fraction', 'data'), textcoords='offset points', color=official_color)
    
    # Change the color of each bar based in his interval
    for i, value in enumerate(values):
        barlist[i].set_facecolor(colors[which_interval(value, parameter)])

In [ ]:
#Assume the user provides the y axis value of interest as a parameter or variable.
y_axis_inputed = 39493.3049414 
update_colors(y_axis_inputed)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy as sp
import scipy.stats

# Use the following data for this assignment:
np.random.seed(12345)

df = pd.DataFrame([np.random.normal(32000, 200000, 3650),
                   np.random.normal(43000, 100000, 3650),
                   np.random.normal(43500, 140000, 3650),
                   np.random.normal(48000, 70000, 3650)],
                  index=[1992, 1993, 1994, 1995])

# Value used for confidence interval comparison
ref_y = 30000


# https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data
def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * sp.stats.t._ppf((1 + confidence) / 2., n - 1)
    return m, m - h, m + h


bar_locations = [1992, 1993, 1994, 1995]
data = []
colors = []

# Determine colors
for i in range(len(bar_locations)):
    mean, low, high = mean_confidence_interval(df.T[bar_locations[i]])
    data.append(mean)
    if low > ref_y:
        color = 'r'
    elif high < ref_y:
        color = 'b'
    elif mean > ref_y:
        diff = 1 - (mean - ref_y) / (high - low)
        color = (1.0, diff, diff)
    else:
        diff = 1 - (ref_y - mean) / (high - low)
        color = (diff, diff, 1.0)

    colors.append(color)

# Create bar plot
plt.bar(bar_locations, data, color=colors, edgecolor="lightgrey")

# Set bar labels
plt.xticks(bar_locations, bar_locations, alpha=0.7)
plt.yticks(alpha=0.7)

# Title
plt.title("95%" + " confidence interval compared to value %1.0f" % ref_y)

# Add y grid for easier reference
ax = plt.gca()
ax.yaxis.grid(True, alpha=0.5)
ax.axhline(y=ref_y)

# Remove the frame of the chart
for spine in plt.gca().spines.values():
    spine.set_visible(False)

# Remove all the ticks on X axis
plt.tick_params(top='off', bottom='off')

plt.show()